# Building LLM applications: Notebook 03

# Tools

## Initialize

In [1]:
import os
import dotenv
import json

from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import tool

from langchain.agents.output_parsers.react_single_input import ReActSingleInputOutputParser

from langchain_ollama import ChatOllama

In [2]:
MODEL = 'llama3.2:3b-instruct-fp16'

In [3]:
# Read fro `.env` file
dotenv.load_dotenv()

OLLAMA_URL = os.getenv('OLLAMA_URL')
print(f"Using Ollama server: {OLLAMA_URL if OLLAMA_URL else 'local'}")

Using Ollama server: http://kqrw311-g5-12xlarge-a.img.astrazeneca.net:8080


## Calling tools manually

### Step 1: Creat tools

In [4]:
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    print(f"Adding {a} + {b}")
    return a + b


@tool
def sub(a: int, b: int) -> int:
    """Substract two numbers"""
    print(f"Substracting {a} - {b}")
    return a - b

### Step 2: Create a list and a map of tools

In [5]:
tools = [add, sub]
tools_map = {t.name.lower(): t for t in tools}
tools_map

{'add': StructuredTool(name='add', description='Add two numbers', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x107539940>),
 'sub': StructuredTool(name='sub', description='Substract two numbers', args_schema=<class 'langchain_core.utils.pydantic.sub'>, func=<function sub at 0x107538d60>)}

### Step 3: Bind tools to the LLM, invoke it

In [6]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

# Bind the tools to the LLM
llm_with_tools = llm.bind_tools(tools)

# Create a message
query = "What is the addition of 40 and 2?"
message = HumanMessage(query)
messages = [message]                # The list of messages to send to the LLM, now is only one message

ret = llm_with_tools.invoke(messages)
ret.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  add (85df0ca5-2d02-4f9e-9221-70d102dd3142)
 Call ID: 85df0ca5-2d02-4f9e-9221-70d102dd3142
  Args:
    a: 40
    b: 2


In [7]:
# Take a look at the tool calls
ret.tool_calls

[{'name': 'add',
  'args': {'a': 40, 'b': 2},
  'id': '85df0ca5-2d02-4f9e-9221-70d102dd3142',
  'type': 'tool_call'}]

### Step 4: Call the functions the LLM suggests

In [8]:
# Call the tools, append the results to the messages list
for tool_call in ret.tool_calls:
    tool_name = tool_call['name'].lower()
    tool = tools_map.get(tool_name)
    if tool:
        tool_msg = tool.invoke(tool_call)
        print(tool_msg)
    print(f"Tool message: {tool_msg}")
    messages.append(tool_msg)

# Show the messages
for m in messages:
    m.pretty_print()

Adding 40 + 2
content='42' name='add' tool_call_id='85df0ca5-2d02-4f9e-9221-70d102dd3142'
Tool message: content='42' name='add' tool_call_id='85df0ca5-2d02-4f9e-9221-70d102dd3142'
================================ Human Message =================================

What is the addition of 40 and 2?
================================= Tool Message =================================
Name: add

42


### Step 5: Append the tool's output, invoke the LLM again

In [9]:
# Invoke the LLM with the messages (including the tool results)
ret = llm_with_tools.invoke(messages)
ret.pretty_print()

================================== Ai Message ==================================

The sum of 40 and 2 is 42.
